In [1]:
from fitness import fitness as fitness_obj
import numpy as np
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.core.problem import ElementwiseProblem
from pymoo.termination import get_termination
from pymoo.optimize import minimize
from tensorflow import keras
from inner_genetic_algorithm import inner_algorithm

In [2]:
class MyProblem(ElementwiseProblem):

    def __init__(self, tf_model):
        n_variables = 5
        super().__init__(
            n_var= n_variables, 
            n_obj=2, 
            n_ieq_constr=0, 
            xl= np.zeros(n_variables), 
            xu=np.ones(n_variables)
        )
        self.tf_model = tf_model
          

    def _evaluate(self, x, out, *args, **kwargs):
        algorithm = inner_algorithm(self.tf_model, x)
        X_res, F_res = algorithm.start()

        with open('results/filters_with_param_X.txt', 'a+') as file:
            np.savetxt(file, X_res)
            file.write('\n')

        with open('results/filters_with_param_F.txt', 'a+') as file:
            np.savetxt(file, F_res)
            file.write('\n')
        
        out["F"] = F_res[0]


In [4]:
model = keras.models.load_model('../cifar10_model_90%val_accuracy.h5')
problem = MyProblem(model)

algorithm = NSGA2(
    pop_size=10,
    n_offsprings=10,
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(prob=0.5 ,eta=20),
    eliminate_duplicates=True
)
termination = get_termination("n_gen", 5)

In [5]:
res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               save_history=False,
               verbose=True)


n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       10 |      3 |             - |             -
     2 |       20 |      7 |  0.000000E+00 |             f
     3 |       30 |      8 |  0.000000E+00 |             f
     4 |       40 |      8 |  0.2000000000 |         ideal
     5 |       50 |     10 |  0.000000E+00 |             f


In [6]:
with open('results/final_results_X.txt', 'w') as file:
    np.savetxt(file, res.X)
with open('results/final_results_F.txt', 'w') as file:
    np.savetxt(file, res.F)


In [7]:
res.X

array([[0.41919451, 0.68780675, 0.28091716, 0.87811744, 0.02738759],
       [0.09834683, 0.42110763, 0.95788953, 0.53316528, 0.69187711],
       [0.09871304, 0.42110763, 0.95788953, 0.53316528, 0.7499139 ],
       [0.09871838, 0.42641878, 0.96234427, 0.57110054, 0.75013624],
       [0.09834015, 0.38686835, 0.96768084, 0.57213216, 0.69210753],
       [0.09871304, 0.42110763, 0.95784977, 0.6086882 , 0.74992198],
       [0.09871838, 0.42641878, 0.9623045 , 0.57213216, 0.75014431],
       [0.09870636, 0.42618278, 0.96236565, 0.57213216, 0.75014431],
       [0.09834683, 0.40466687, 0.95788953, 0.53316528, 0.69187711],
       [0.09834015, 0.42618278, 0.96236565, 0.57213216, 0.69210753]])

In [8]:
res.F

array([[0.25, 0.35],
       [0.5 , 0.25],
       [0.5 , 0.25],
       [0.5 , 0.25],
       [0.5 , 0.25],
       [0.5 , 0.25],
       [0.5 , 0.25],
       [0.5 , 0.25],
       [0.5 , 0.25],
       [0.5 , 0.25]])